In [1]:
from drain.model import Drain,Node,LogCluster
from drain.optimizer import Optimizer
from drain.plotter import createPlot
from drain.partition import Partition
from utils.data_utils import *
import unsupervised.model as unsupervised_model
# import unsupervised.model as unsupervised_model
from torch import nn,optim
from torch.utils.data import  DataLoader

import pandas as pd
import os

In [100]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 32
batch_size = 2048
input_size = 1
window_size = 10
num_layers = 2
hidden_size = 64

In [101]:
logPath = 'D:\\毕业设计\\loghub\\HDFS_1\\HDFS.log'
parsed_result = '.\\data\\parser\\log_item_to_label.csv'
cluster_result = '.\\data\\parser\\'
# partition
partition_output = '.\\data\\lstm\\dataset\\'
instance_file = 'instance.csv'
label_file = '.\\data\\partition\\anormaly_label.csv'
normal_output = 'normal.csv'
abnormal_output = 'abnormal.csv'

#lstm dataset
lstm_dataset = '.\\data\\lstm\\dataset\\'
unsupervised_train = 'train.csv'
unsupervised_test_normal = 'normal.csv'
unsupervised_test_abnormal = 'abnormal.csv'

predicted_input = 'unsupervised_input.csv'
predicted_normal_output = 'unsupervised_normal_output.csv'
predicted_abnormal_output = 'unsupervised_abnormal_output.csv'
supervised_test = 'supervised_test.csv'

In [102]:
def parse_log():

    rex = ['blk_(|-)[0-9]+', '(/|)([0-9]+\.){3}[0-9]+(:[0-9]+|)(:|)']
    removeCol = [0,1,2]
    myParser = Drain(rex=rex,removeCol=removeCol,st=0.4)
    myParser.fit(isReconstruct=True,inputFile=logPath,outputFile=parsed_result)

    myParser.save(savePath=cluster_result)

'''
partition phase
'''
def partition():

    partition = Partition(outputFileDir=partition_output,log_item_to_event_id_file=parsed_result,instances_file_path=instance_file,label_file_path=label_file,normal_output=normal_output,abnormal_output=abnormal_output)
    partition.partition_by_file()
    partition.map_log_seq_to_label()

    split_data(input_dir=partition_output,output_dir=lstm_dataset)

In [122]:
model_dir = 'model'
version = '0.1'
input_size = 1
model_usl_name = 'type={}_version={}'.format('unsupervised',version)


In [123]:

model = unsupervised_model.Model(input_size, hidden_size, num_layers, num_classes).to(device)

if os.path.exists(model_dir + '/' + model_usl_name + '.pt'):
    model.load_state_dict(torch.load(model_dir + '/' + model_usl_name + '.pt'))
    print('loading model {} success.'.format(model_usl_name))

loading model type=unsupervised_version=0.1 success.


In [104]:
lstm_dataset + unsupervised_train

'.\\data\\lstm\\dataset\\train.csv'

In [120]:
criterion = nn.CrossEntropyLoss(ignore_index=31)
optimizer = optim.Adam(model.parameters())
train_dataset = generate_train_data(lstm_dataset + unsupervised_train)
dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

loading data: 100%|█████████████████████████████████████████████████████████████| 5582/5582 [00:00<00:00, 19583.29it/s]


Number of sessions(.\data\lstm\dataset\train.csv): 5582
Number of seqs(.\data\lstm\dataset\train.csv): 114877


In [121]:
model.train()
unsupervised_model.train(model, dataloader, criterion, optimizer, current_epoch=0, num_epochs=30)

Epoch [1/30], train_loss: 0.4358


KeyboardInterrupt: 

In [108]:
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)
torch.save(model.state_dict(), model_dir + '/' + model_usl_name + '.pt')
print('Finished Training')

Finished Training


In [118]:
model.eval()
batch_size = 20000
test_normal_session, test_normal_dataset, normal_block = generate_predicted_data(
    lstm_dataset + unsupervised_test_normal, window_size)
normal_dataloader = DataLoader(test_normal_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
test_abnormal_session, test_abnormal_dataset, abnormal_block = generate_predicted_data(
    lstm_dataset + unsupervised_test_abnormal, window_size)
abnormal_dataloader = DataLoader(test_abnormal_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

.\data\lstm\dataset\normal.csv: 100%|██████████████████████████████████████████| 14176/14176 [00:01<00:00, 8591.60it/s]


Number of sessions(.\data\lstm\dataset\normal.csv): 14176
Number of seqs(.\data\lstm\dataset\normal.csv): 269279


.\data\lstm\dataset\abnormal.csv: 100%|█████████████████████████████████████████| 4124/4124 [00:00<00:00, 11552.76it/s]


Number of sessions(.\data\lstm\dataset\abnormal.csv): 4124
Number of seqs(.\data\lstm\dataset\abnormal.csv): 88592


In [6]:
model.eval()
batch_size = 20000
test_normal_session, test_normal_dataset, normal_block = generate_predicted_data(
    lstm_dataset + 'no_dup_normal.csv', window_size)
normal_dataloader = DataLoader(test_normal_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
test_abnormal_session, test_abnormal_dataset, abnormal_block = generate_predicted_data(
    lstm_dataset + 'no_dup_abnormal.csv', window_size)
abnormal_dataloader = DataLoader(test_abnormal_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

.\data\lstm\dataset\no_dup_normal.csv: 100%|██████████████████████████████████| 14176/14176 [00:01<00:00, 11101.14it/s]


Number of sessions(.\data\lstm\dataset\no_dup_normal.csv): 14176
Number of seqs(.\data\lstm\dataset\no_dup_normal.csv): 269279


.\data\lstm\dataset\no_dup_abnormal.csv: 100%|██████████████████████████████████| 4124/4124 [00:00<00:00, 10938.86it/s]


Number of sessions(.\data\lstm\dataset\no_dup_abnormal.csv): 4124
Number of seqs(.\data\lstm\dataset\no_dup_abnormal.csv): 88592


In [7]:
import unsupervised.predictor as unsupervised_predictor

In [8]:
re = unsupervised_predictor.predict(model, abnormal_dataloader,test_abnormal_session,10, window_size,ts=0.0005)

loading data: 5it [00:24,  4.80s/it]


In [9]:
re.count(False)

236

In [131]:
test_normal_result, test_abnormal_result = unsupervised_predictor.count_metries(model, normal_dataloader, abnormal_dataloader,
                                                        test_normal_session, test_abnormal_session,
                                                        10, window_size,ts=0.0001)


loading data: 0it [00:00, ?it/s]
loading data: 1it [00:04,  4.76s/it]
loading data: 2it [00:09,  4.69s/it]
loading data: 3it [00:13,  4.68s/it]
loading data: 4it [00:18,  4.71s/it]
loading data: 5it [00:23,  4.83s/it]
loading data: 6it [00:29,  5.01s/it]
loading data: 7it [00:34,  5.07s/it]
loading data: 8it [00:39,  4.96s/it]
loading data: 9it [00:44,  4.94s/it]
loading data: 10it [00:48,  4.79s/it]
loading data: 11it [00:53,  4.78s/it]
loading data: 12it [00:58,  4.81s/it]
loading data: 13it [01:02,  4.77s/it]
loading data: 14it [01:04,  4.64s/it]

loading data: 0it [00:00, ?it/s]
loading data: 1it [00:04,  4.87s/it]
loading data: 2it [00:09,  4.79s/it]
loading data: 3it [00:15,  5.26s/it]
loading data: 4it [00:20,  5.06s/it]
loading data: 5it [00:23,  4.63s/it]

elapsed_time: 88.259s
false positive (FP): 862, false negative (FN): 698, Precision: 79.897%, Recall: 83.075%, F1-measure: 81.455%
Finished Predicting


In [110]:
test_normal_result, test_abnormal_result = unsupervised_predictor.count_metries(model, normal_dataloader, abnormal_dataloader,
                                                        test_normal_session, test_abnormal_session,
                                                        10, window_size,ts=0)

loading data: 14it [01:12,  5.19s/it]
loading data: 5it [00:21,  4.35s/it]

elapsed_time: 94.541s
false positive (FP): 396, false negative (FN): 1540, Precision: 86.711%, Recall: 62.658%, F1-measure: 72.748%
Finished Predicting


In [136]:
predicted_input

'unsupervised_input.csv'

In [125]:
sessions, dataset, block_to_seqs = generate_predicted_data(lstm_dataset+predicted_input,window_size)


.\data\lstm\dataset\unsupervised_input.csv: 100%|████████████████████████████████| 5000/5000 [00:00<00:00, 6385.73it/s]

Number of sessions(.\data\lstm\dataset\unsupervised_input.csv): 5000
Number of seqs(.\data\lstm\dataset\unsupervised_input.csv): 92307


In [126]:
batch_size = 20000
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

In [127]:
normal_result = unsupervised_predictor.predict(model, dataloader,sessions,10, window_size,ts=0.0005)


loading data: 0it [00:00, ?it/s]
loading data: 1it [00:05,  5.22s/it]
loading data: 2it [00:10,  5.27s/it]
loading data: 3it [00:16,  5.37s/it]
loading data: 4it [00:21,  5.46s/it]
loading data: 5it [00:25,  5.04s/it]


In [116]:
def show_result(result):
    TN = result[:3000].count(False)
    FN = result[3000:].count(False)
    print('TN:{} FN:{}'.format(TN,FN))
    print('Negative Precision:',TN/(TN+FN))
    TP = result[3000:].count(True)
    FP = result[:3000].count(True)
    print('TP:{} FP:{}'.format(TP,FP))
    print('Positive Precision:',TP/(TP+FP))

In [128]:
show_result(normal_result)

TN:2647 FN:140
Negative Precision: 0.9497667743092931
TP:1860 FP:353
Positive Precision: 0.8404880253050158


In [118]:
abnormal_result = unsupervised_predictor.predict(model, dataloader,sessions,10, window_size,ts=0)

loading data: 57it [00:13,  4.28it/s]


In [119]:
show_result(abnormal_result)

TN:5 FN:7
Negative Precision: 0.4166666666666667
TP:1993 FP:2995
Positive Precision: 0.3995589414595028


In [30]:
f = open(lstm_dataset+predicted_normal_output,'w')
f.write('BlockId,EventSequence\n')
for i,label in enumerate(normal_result):
    if label==False:
        f.write(block_to_seqs[i][0]+',"'+str(block_to_seqs[i][1])+'"\n')
f.close()

In [31]:
f = open(lstm_dataset+predicted_abnormal_output,'w')
f.write('BlockId,EventSequence\n')
for i,label in enumerate(abnormal_result):
    if label:
        f.write(block_to_seqs[i][0]+',"'+str(block_to_seqs[i][1])+'"\n')
f.close()

In [32]:
input_size = 768

In [33]:
lstm_dataset+predicted_normal_output

'.\\data\\lstm\\dataset\\unsupervised_normal_output.csv'

In [34]:
train_dataset, test_dataset=generate_bert_data(lstm_dataset,predicted_normal_output,predicted_abnormal_output,"./data/lstm/bert_cache.pth")

abnormal:: 100%|███████████████████████████████████████████████████████████████████| 1324/1324 [00:33<00:00, 39.34it/s]


Number of sessions(.\data\lstm\dataset\): 4110
Number of normal sessions: 2786
Number of abnormal sessions: 1324


In [35]:
batch_size = 100
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [36]:
import supervised.model as supervised_model

In [37]:
model = supervised_model.Model(input_size, hidden_size, num_layers, num_classes).to(device)

In [38]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [39]:
model_sl_name = 'type={}_version={}'.format('supervised',version)

In [40]:
if os.path.exists(model_dir + '/' + model_sl_name + '.pt'):
    print('loading model {} success.'.format(model_sl_name))
    model.load_state_dict(torch.load(model_dir + '/' + model_name + '.pt'))

In [86]:
torch.cuda.empty_cache()

In [97]:
model.train()
supervised_model.train(model,train_dataloader,criterion,optimizer,current_epoch=0,num_epochs=50,input_size=input_size)

Epoch [1/50], train_loss: 0.0735
Epoch [2/50], train_loss: 0.0711
Epoch [3/50], train_loss: 0.0734
Epoch [4/50], train_loss: 0.0697
Epoch [5/50], train_loss: 0.0715
Epoch [6/50], train_loss: 0.0770
Epoch [7/50], train_loss: 0.0797
Epoch [8/50], train_loss: 0.0651
Epoch [9/50], train_loss: 0.0617
Epoch [10/50], train_loss: 0.0633
Epoch [11/50], train_loss: 0.0622
Epoch [12/50], train_loss: 0.0681
Epoch [13/50], train_loss: 0.0932
Epoch [14/50], train_loss: 0.1097
Epoch [15/50], train_loss: 0.0853
Epoch [16/50], train_loss: 0.0812
Epoch [17/50], train_loss: 0.0833
Epoch [18/50], train_loss: 0.0690
Epoch [19/50], train_loss: 0.0655
Epoch [20/50], train_loss: 0.0703
Epoch [21/50], train_loss: 0.0736
Epoch [22/50], train_loss: 0.0646
Epoch [23/50], train_loss: 0.0844
Epoch [24/50], train_loss: 0.0883
Epoch [25/50], train_loss: 0.1821
Epoch [26/50], train_loss: 0.1909
Epoch [27/50], train_loss: 0.1072
Epoch [28/50], train_loss: 0.0690
Epoch [29/50], train_loss: 0.0622
Epoch [30/50], train_lo

In [65]:
import numpy as np
def accuracy(y_pred, y_true):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    pred = np.argmax(y_pred.cpu().numpy(),1)
    for i,val in enumerate(pred):
        if val == 0:
            if y_true[i]==0:
                TN +=1
            else:
                FN +=1
        else:
            if y_true[i]==1:
                TP+=1
            else:
                FP+=1
    return TP,TN,FP,FN

In [75]:
def predict_result(model,data):
    model.eval()
    with torch.no_grad():
        epoch_loss = 0
        for step, (seq, label) in enumerate(data):
            seq = seq.clone().detach().view(-1, seq.shape[1], input_size).to(device)
            test_output = model(seq.to(device))
            if step == 0:
                output = test_output
                labels = label
            else:
                labels = torch.cat([labels, label], 0)
                output = torch.cat([output, test_output], 0)
            epoch_loss += criterion(test_output, label.to(device)).data
        TP,TN,FP,FN = accuracy(output, labels)
        epoch_accuracy = (TP+TN)/(FP+FN+TP+TN)
        epoch_loss = epoch_loss / len(data)
        print('TN:{} FN:{}'.format(TN,FN))
        print('Negative Precision:',round(TN/(TN+FN),3))
        print('TP:{} FP:{}'.format(TP,FP))
        print('Positive Precision:',round(TP/(TP+FP),3))
        print('loss: ', round(epoch_loss.item(), 3), 'accuracy: ', round(epoch_accuracy, 3))

In [93]:
predict_result(model,train_dataloader)

TN:1643 FN:52
Negative Precision: 0.969
TP:758 FP:13
Positive Precision: 0.983
loss:  0.088 accuracy:  0.974


In [94]:
predict_result(model,test_dataloader)

TN:1105 FN:38
Negative Precision: 0.967
TP:476 FP:25
Positive Precision: 0.95
loss:  0.134 accuracy:  0.962


In [56]:
supervised_input_normal = 'supervised_input_normal.csv'
supervised_input_abnormal = 'supervised_input_abnormal.csv'

In [61]:
test_dataset1,test_dataset2 =generate_bert_data(lstm_dataset,supervised_input_normal,supervised_input_abnormal,"./data/lstm/bert_cache.pth")

abnormal:: 100%|███████████████████████████████████████████████████████████████████| 2115/2115 [00:52<00:00, 40.56it/s]

Number of sessions(.\data\lstm\dataset\): 7092
Number of normal sessions: 4977
Number of abnormal sessions: 2115


In [78]:
test_dataloader1 = DataLoader(test_dataset1, batch_size=batch_size, shuffle=True)
test_dataloader2 = DataLoader(test_dataset2, batch_size=batch_size)

In [98]:
predict_result(model,test_dataloader1)

TN:2757 FN:201
Negative Precision: 0.932
TP:1063 FP:234
Positive Precision: 0.82
loss:  0.416 accuracy:  0.898


In [99]:
predict_result(model,test_dataloader2)

TN:1800 FN:127
Negative Precision: 0.934
TP:724 FP:186
Positive Precision: 0.796
loss:  0.456 accuracy:  0.89
